## Load dataset

Below you can see code used to load the dataset of 15 3D models. Each sample consists of a 1024x1024 or 2048x2048 texture, a 3D model as a .obj file, and a list of ground truth labels. Some models only have one label, while other have multiple ones.

In [1]:
import os
import random
import csv

In [2]:
class Model3D:
    def __init__(self, folder, data_dir):
        self.name = folder
        absolute_model_path = os.path.join(data_dir, self.name)

        self.texture_path = Model3D._get_texture_path(absolute_model_path)
        self.obj_path = os.path.join(absolute_model_path, "{}.obj".format(self.name))
        self.labels = Model3D._load_labels(absolute_model_path)

    def __str__(self):
        return "{}: labels {}, texture {}, obj {}".format(self.name, self.labels, self.texture_path, self.obj_path)

    @staticmethod
    def _get_texture_path(path):
        """Determines if texture is a jpg or png file, and returns absolute path to texture file.

        Parameters
        ----------
        path : String
            Absolute path to dataset sample folder.

        Returns
        -------
        String
            Absolute path to texture file.
        """
        if not os.path.isdir(path):
            raise ValueError("The given absolute path is not a directory!")

        for file in os.listdir(path):
            if file.endswith(".jpg"):
                return os.path.join(path, file)
            elif file.endswith(".png"):
                return os.path.join(path, file)

        raise ValueError("No jpg or png files found in the given directory!")

    @staticmethod
    def _load_labels(path):
        """Reads labels of a certain sample from the dataset and returns them.

        Parameters
        ----------
        path : String
            Absolute path to dataset sample folder.

        Returns
        -------
        String or list
            Returns a list of integers, or if this is the dog model, just returns "dog" as a label.
        """
        if not os.path.isdir(path):
            raise ValueError("The given absolute path is not a directory!")

        labels_file_path = os.path.join(path, "labels.txt")
        try:
            labels_file = open(labels_file_path)
        except FileNotFoundError:
            raise FileNotFoundError("No txt files found in the given path! Can not find labels!")

        # labels are written only on the first line of the file, we only read the first line
        labels = next(csv.reader(labels_file, delimiter=','))
        # German shepherd model has all 120+ dog labels as true labels, that is encoded only as "dog" to save
        # make things easier
        if labels[0] == 'dog':
            return labels[0]
        else:
            try:
                int_labels = [int(label) for label in labels]
                return int_labels
            except ValueError as e:
                print("Original exception message: {}".format(str(e)))
                raise ValueError("A label of {} does not represent an int!".format(path))
            finally:
                labels_file.close()


def get_object_folders(data_dir):
    """Returns a list of all folders in the given folder.

    Parameters
    ----------
    path : String
        Absolute path to dataset sample folder.

    Returns
    -------
    List of strings
        Returns a list of with the name of each folder.
    """
    if not os.path.isdir(data_dir):
        raise ValueError("The given data path is not a directory!")

    return [folder for folder in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, folder))]


def load_dataset(data_dir):
    object_folders = get_object_folders(data_dir)
    models = [Model3D(folder, data_dir) for folder in object_folders]
    for model in models:
        print(str(model))

    return models

Define code to generate random target labels, different than the true labels of the model.

In [3]:
def get_random_target_label(ground_truth_labels):
    label_set = set(ground_truth_labels)

    # loop until we have a random target label distinct from the true labels
    while True:
        target_label = random.randint(0, 999)

        # dog model has al 120+ dog breeds as true labels, so we need to check if the label is outside that range
        if ground_truth_labels == "dog":
            if target_label < 151 or target_label > 275:
                return target_label
        # just check that the chosen target is not in the set of true labels
        elif target_label not in label_set:
            return target_label

In [4]:
import config

In [ ]:
models = load_dataset("./dataset")

for model in models:
    # we only run algorithm on models representing objects that the authors also used in their dataset
    if model.name not in ["crocodile", "jeep", "orca", "rugby_ball", "running_shoe"]:
        config.OBJ_PATH = model.obj_path
        config.TEXTURE_PATH = model.texture_path
        config.NAME = model.name
        
        print("Creating adversarial examples for model {}".format(model.name))
        for i in range(5):
            config.TARGET_LABEL = get_random_target_label(model.labels)
            print("Target label {}".format(config.TARGET_LABEL))
            
            # run script to create adversarial texture for the given target
            %run main.py
        

barrel: labels [427], texture ./dataset\barrel\barrel.jpg, obj ./dataset\barrel\barrel.obj
baseball: labels [429], texture ./dataset\baseball\baseball.png, obj ./dataset\baseball\baseball.obj
camaro: labels [817, 436, 751], texture ./dataset\camaro\camaro.png, obj ./dataset\camaro\camaro.obj
clownfish: labels [393], texture ./dataset\clownfish\clownfish.jpg, obj ./dataset\clownfish\clownfish.obj
crocodile: labels [49, 50], texture ./dataset\crocodile\crocodile.jpg, obj ./dataset\crocodile\crocodile.obj
german_shepherd: labels dog, texture ./dataset\german_shepherd\german_shepherd.png, obj ./dataset\german_shepherd\german_shepherd.obj
jeep: labels [609, 586, 408], texture ./dataset\jeep\jeep.png, obj ./dataset\jeep\jeep.obj
orange: labels [950], texture ./dataset\orange\orange.jpg, obj ./dataset\orange\orange.obj
orca: labels [148], texture ./dataset\orca\orca.png, obj ./dataset\orca\orca.obj
purse: labels [748, 893], texture ./dataset\purse\purse.png, obj ./dataset\purse\purse.obj
rugb